<a href="https://colab.research.google.com/github/xN1ckuz/Crosswalks-Detection-using-YoloV5/blob/main/crosswalks_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

folder = 'project_VeP'#@param{type:"string"}

drive.mount('/content/gdrive')
print('Connecting to Drive....')

%cd /content/gdrive/MyDrive/$folder/
print('Done!')

In [ ]:
!git clone https://github.com/ultralytics/yolov5
!pip install -r yolov5/requirements.txt
import torch
print("Repo Clone Completed!")

In [ ]:
epoch = 250 #@param {type:"slider", min:100, max:1000, step:10}
training_option = 'Train new CNN' #@param ["Train new CNN", "Resume an interrupted Training"]
weight = 'yolov5m.pt' #@param["yolov5s.pt","yolov5m.pt","yolov5l.pt","yolov5x.pt"]
batch = 64 #@param {type:"slider", min:1, max:64, step:1}

if training_option == 'Train new CNN':
    
    %rm -r /content/gdrive/MyDrive/$folder/training_results/exp
    !python /content/gdrive/MyDrive/$folder/yolov5/train.py --img 512 --batch $batch --epochs $epoch --data /content/gdrive/MyDrive/$folder/training_dataset/data.yaml --weights $weight --cache --project /content/gdrive/MyDrive/$folder/training_results/
else :
    !python /content/gdrive/MyDrive/$folder/yolov5/train.py --resume /content/gdrive/MyDrive/$folder/training_results/export/weights/last.pt 

In [ ]:
%rm -r /content/gdrive/MyDrive/$folder/test_results/exp
!python /content/gdrive/MyDrive/$folder/yolov5/val.py --data /content/gdrive/MyDrive/$folder/test_dataset/data.yaml --weights /content/gdrive/MyDrive/$folder/training_results/exp/weights/best.pt --project /content/gdrive/MyDrive/$folder/test_results/

In [ ]:
#Printing result to confront
from PIL import Image

precision_training = '/content/gdrive/MyDrive/'+folder+'/training_results/exp/P_curve.png'
precision_test = '/content/gdrive/MyDrive/'+folder+'/test_results/exp/P_curve.png'

im1 = Image.open(precision_training)
im2 = Image.open(precision_test)
print("Precision Training")
display(im1)
print("Precision Test")
display(im2)

In [ ]:
video_name = 'video1.mp4' #@param {type:"string"}
!python /content/gdrive/MyDrive/$folder/yolov5/detect.py --source /content/gdrive/MyDrive/$folder/videos/$video_name --weights /content/gdrive/MyDrive/$folder/training_results/exp/weights/best.pt --project /content/gdrive/MyDrive/$folder/detection_results/
                            